In [12]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

driver_path = "./msedgedriver.exe"

service = EdgeService(executable_path=driver_path)
driver = webdriver.Edge(service=service)

driver.get("https://www.genesis.com/kr/ko/support/faq.html")

In [9]:
r=requests.post("https://www.google-analytics.com/j/collect?v=1&_v=j102&a=1512356276&t=pageview&_s=1&dl=https%3A%2F%2Fwww.genesis.com%2Fkr%2Fko%2Fsupport%2Ffaq%2Fwebsite%2Ffaq_tab.html&ul=ko-kr&dt=%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4%20FAQ%20-%20%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4%20%ED%99%88%ED%8E%98%EC%9D%B4%EC%A7%80%20%EA%B4%80%EB%A0%A8%20%EA%B6%81%EA%B8%88%ED%95%9C%20%EC%A0%90%EC%97%90%20%EB%8C%80%ED%95%9C%20%EB%8B%B5%EB%B3%80%EC%9D%84%20%EC%B0%BE%EC%95%84%EB%B3%B4%EC%84%B8%EC%9A%94%20%7C%20%7C%20%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4&sr=1536x864&vp=779x695&_u=QCCAAEABAAAAACAAI~&jid=&gjid=&cid=1783759609.1768464161&tid=UA-80300375-2&_gid=1686350376.1768464161&_slc=1&gtm=45He61d1n71WHPSRBv72617950za200zd72617950&gcd=13l3l3l3l1l1&dma=0&tag_exp=103116026~103200004~104527907~104528501~104684208~104684211~105391253~115938465~115938468~117041588&z=1369120179")

In [10]:
r.status_code

200

In [11]:
r.text

'2,cG-YXHDJJ92KE'

In [14]:
wait = WebDriverWait(driver, 10)

# 버튼 개수 먼저 파악
tabs = wait.until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "cp-faq__tab-link"))
)

tab_count = len(tabs)

for i in range(tab_count):
    # 🔥 DOM 갱신 대비: 매번 다시 찾기
    tabs = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "cp-faq__tab-link"))
    )

    tab = tabs[i]

    # 화면에 보이도록 스크롤
    driver.execute_script("arguments[0].scrollIntoView(true);", tab)
    time.sleep(0.3)

    # 클릭
    tab.click()
    time.sleep(1)  # 탭 전환/데이터 로딩 대기


ElementNotInteractableException: Message: element not interactable
  (Session info: MicrosoftEdge=143.0.3650.139); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff609dc8815
	0x7ff609d38e84
	0x7ff609b4c746
	0x7ff609b94063
	0x7ff609b8a9e6
	0x7ff609bb23ea
	0x7ff609b8a2e5
	0x7ff609b8a1fd
	0x7ff609b8a2e5
	0x7ff609bcc8de
	0x7ff609b8982a
	0x7ff609b88b33
	0x7ff609b89653
	0x7ff609c722e4
	0x7ff609c8109c
	0x7ff609c7ac7f
	0x7ff609e59b77
	0x7ff609d446e6
	0x7ff609d3eaf4
	0x7ff609d3ec39
	0x7ff609d32cfd
	0x7ffc6312e8d7
	0x7ffc646ec53c


In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from urllib.parse import urljoin
from selenium.webdriver.edge.service import Service as EdgeService
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests

def clean_text(s: str) -> str:
    if not s:
        return ""
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"\n\s+", "\n", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    s = re.sub(r"[ \t]{2,}", " ", s)
    return s.strip()

def scrape_faq(driver, base_url: str):
    wait = WebDriverWait(driver, 10)
    results = []

    # 탭 개수 파악
    tab_elems = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".cp-faq__tab-link"))
    )
    tab_count = len(tab_elems)

    for ti in range(tab_count):
        # DOM 갱신 대비: 매번 다시 찾고 i번째 클릭
        tab_elems = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".cp-faq__tab-link"))
        )
        tab = tab_elems[ti]

        # 탭 이름(가능하면)
        tab_name = clean_text(tab.text)

        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tab)
        time.sleep(0.2)
        driver.execute_script("arguments[0].click();", tab)

        # 탭 클릭 후 FAQ 리스트 로딩 대기
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cp-faq__accordion-list")))

        # 아코디언 아이템들
        items = driver.find_elements(By.CSS_SELECTOR, ".cp-faq__accordion-list .cp-faq__accordion-item")
        item_count = len(items)

        # 각 아코디언을 순차적으로 열고 내용 추출
        for ai in range(item_count):
            # DOM 갱신 대비: 다시 찾기
            items = driver.find_elements(By.CSS_SELECTOR, ".cp-faq__accordion-list .cp-faq__accordion-item")
            item = items[ai]

            # 토글 버튼(a.accordion-btn) 찾기
            btn = item.find_element(By.CSS_SELECTOR, "a.accordion-btn")

            # 질문/카테고리
            try:
                category = clean_text(item.find_element(By.CSS_SELECTOR, "strong.accordion-label").text)
            except:
                category = ""
            try:
                question = clean_text(item.find_element(By.CSS_SELECTOR, "p.accordion-title").text)
            except:
                question = btn.get_attribute("title") or ""

            # 현재 열려있는지 확인
            is_active = "is-active" in (item.get_attribute("class") or "")
            if not is_active:
                # 클릭해서 열기 (span.ico-accordion이 아니라 a.accordion-btn 클릭이 안정적)
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
                time.sleep(0.15)
                driver.execute_script("arguments[0].click();", btn)

                # 열릴 때까지 대기 (is-active 되거나 panel display:block)
                wait.until(lambda d: "is-active" in (items[ai].get_attribute("class") or ""))

            # 열린 상태에서 panel 내용 수집
            # (다시 찾기: stale 방지)
            items = driver.find_elements(By.CSS_SELECTOR, ".cp-faq__accordion-list .cp-faq__accordion-item")
            item = items[ai]

            panel_inner = item.find_element(By.CSS_SELECTOR, ".accordion-panel .accordion-panel-inner")

            # 답변 텍스트
            answer_text = clean_text(panel_inner.text)

            # 이미지들
            imgs = panel_inner.find_elements(By.CSS_SELECTOR, "img")
            image_list = []
            for img in imgs:
                src = img.get_attribute("src") or ""
                alt = img.get_attribute("alt") or ""
                # 상대경로 대비
                full_src = urljoin(base_url, src)
                image_list.append({"src": full_src, "alt": alt})

            results.append({
                "tab": tab_name,
                "category": category,
                "question": question,
                "answer": answer_text,
                "images": image_list
            })

    return results


def pretty_print(records, max_answer_chars=600):
    for i, r in enumerate(records, 1):
        print(f"\n{'='*80}")
        print(f"[{i}] 탭: {r['tab']}")
        if r["category"]:
            print(f"카테고리: {r['category']}")
        print(f"Q. {r['question']}")
        print("-"*80)
        ans = r["answer"]
        if max_answer_chars and len(ans) > max_answer_chars:
            ans = ans[:max_answer_chars] + "…"
        print(ans if ans else "(답변 텍스트 없음)")
        if r["images"]:
            print("\n[이미지]")
            for img in r["images"]:
                print(f"- {img['alt']} | {img['src']}")
        print(f"{'='*80}")


# ---------------- 실행 예시 ----------------
if __name__ == "__main__":
    url = "https://www.genesis.com/kr/ko/support/faq.html"   # ✅ 여기에 실제 URL
    driver_path = "./msedgedriver.exe"

    service = EdgeService(executable_path=driver_path)
    driver = webdriver.Edge(service=service)    
    driver.get(url)

    data = scrape_faq(driver, base_url=url)
    pretty_print(data)

    driver.quit()



[1] 탭: ALL
카테고리: [차량 구매]
Q. 제네시스 구입 후 세금계산서 발급받으려고 하는데 어떻게 해야 하나요?
--------------------------------------------------------------------------------
[개인사업자 및 법인사업자 구입고객]
2010년 7월 1일부터 세금계산서가 서면이 아닌 전자 세금계산서로 발행이 됩니다. 사이트( http://e-vat.hyundai.com ) 에서 발급받으시면 됩니다.
[개인 구입고객]
2011년 4월11일부터 차량 구입하신 지점(대리점)에서 영수증이 발행됩니다.
※재발행은 차량 구입하신 지점(대리점)으로 문의해 주시기 바랍니다.

[2] 탭: ALL
카테고리: [차량 구매]
Q. 제네시스 차량 구입 시 의무보험료란 금액을 납부하였습니다. 이것은 어떤 종류의 보험인가요?
--------------------------------------------------------------------------------
의무보험은 차량이 출고센터를 동해 출고되어 임시운행 기간 동안에만 유효한 보험입니다. 이는 임시운행 기간 중 발생되는 교통사고 피해자의 피해배상 보장을 위한 자동차 손해배상보장법 개정으로 임시운행허가 차량의 의무보험 가입이 법제화 되었습니다. 보상책임기간은 출고일부터 임시운행허가 기간인 10일(기간 만료일 24시까지)동안이고 보험료는 차종별 정액보험료가 적용됩니다. (대부분의 경우 1만원 이내) 따라서 차량을 구입하신 고객께서는 임시운행허가기간 종료 이전에 1년만기 책임보험, 혹은 종합보험에 반드시 가입하셔야 합니다. 의무보험은 1회 소멸성 보험으로 임시운행증을 반납하거나 다른 책임보험에 가입하시는 경우 자동으로 해지가 됩니다.

[3] 탭: ALL
카테고리: [차량 구매]
Q. 일정기간 이상 지난 차량은 자동차세가 경감된다고 하는데 그 내용을 알고 싶습니다.
----------------------------------------------------

In [21]:
import pandas as pd
data =pd.DataFrame(data)

In [26]:
data.to_pickle('genesis.pkl')
data.to_csv("genesis.csv",encoding='utf-8')